In [1]:
import pandas
from matplotlib import pyplot

%matplotlib inline

In [2]:
teams = pandas.read_csv('Teams.csv')
regular_season = pandas.read_csv('RegularSeasonDetailedResults.csv')


In [5]:
regular_season.columns

Index([u'Season', u'Daynum', u'Wteam', u'Wscore', u'Lteam', u'Lscore', u'Wloc',
       u'Numot', u'Wfgm', u'Wfga', u'Wfgm3', u'Wfga3', u'Wftm', u'Wfta',
       u'Wor', u'Wdr', u'Wast', u'Wto', u'Wstl', u'Wblk', u'Wpf', u'Lfgm',
       u'Lfga', u'Lfgm3', u'Lfga3', u'Lftm', u'Lfta', u'Lor', u'Ldr', u'Last',
       u'Lto', u'Lstl', u'Lblk', u'Lpf'],
      dtype='object')

In [7]:
frames = []
for season in regular_season.Season.unique():
    team_season = teams.copy()
    team_season['season'] = season
    team_season.index = team_season.Team_Id.apply(str) + team_season.season.apply(str)
    frames.append(team_season)

team_seasons = pandas.concat(frames)
print(team_seasons.head())


          Team_Id    Team_Name  season
11012003     1101  Abilene Chr    2003
11022003     1102    Air Force    2003
11032003     1103        Akron    2003
11042003     1104      Alabama    2003
11052003     1105  Alabama A&M    2003


In [9]:
loser_columns = ['Lscore', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3','Lftm', 'Lfta', 'Lor', 'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk', 'Lpf']
winner_columns = ['Wscore', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wor', 'Wdr', 'Wast', 'Wto', 'Wstl', 'Wblk', 'Wpf']
fixed_columns = [column[1:] for column in winner_columns]



In [13]:
season_averages = []
for index, row in team_seasons.iterrows():
    wins = regular_season.Wteam == row.Team_Id
    loses = regular_season.Lteam == row.Team_Id
    right_season = regular_season.Season == row.season

    won_games = regular_season[right_season & wins][winner_columns]
    won_games.columns = fixed_columns

    lost_games = regular_season[right_season & loses][loser_columns]
    lost_games.columns = fixed_columns

    games = pandas.concat([won_games, lost_games]).mean()
    games['Team_Id'] = row.Team_Id
    games['Season'] = row.season
    games['Team_Name'] = row.Team_Name
    games = games.to_frame().transpose()

    season_averages.append(games)

season_averages = pandas.concat(season_averages).dropna()


In [14]:
season_averages.head()

,score,fgm,fga,fgm3,fga3,ftm,fta,or,dr,ast,to,stl,blk,pf,Team_Id,Season,Team_Name
0,57.25,19.1429,39.7857,7.82143,20.8214,11.1429,17.1071,4.17857,16.8214,13,11.4286,5.96429,1.78571,18.75,1102,2003,Air Force
0,78.7778,27.1481,55.8519,5.44444,16.0741,19.037,25.8519,9.77778,19.9259,15.2222,12.6296,7.25926,2.33333,19.8519,1103,2003,Akron
0,69.2857,24.0357,57.1786,6.35714,19.8571,14.8571,20.9286,13.5714,23.9286,12.1071,13.2857,6.60714,3.78571,18.0357,1104,2003,Alabama
0,71.7692,24.3846,61.6154,7.57692,20.7692,15.4231,21.8462,13.5,23.1154,14.5385,18.6538,9.30769,2.07692,20.2308,1105,2003,Alabama A&M
0,63.6071,23.4286,55.2857,6.10714,17.6429,10.6429,16.4643,12.2857,23.8571,11.6786,17.0357,8.35714,3.14286,18.1786,1106,2003,Alabama St


In [15]:
season_averages.to_csv("SeasonAverages.csv", index=False)